In [ ]:
1+1

In [2]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")
using DataStructures
using DataStructuresExtended
using JLD
using DataFrames

In [ ]:
OUTPUT_DIR = "../../documents_prepared/Journal\ Papers/SumResynthesis/data/"

In [ ]:
`git add ../../documents_prepared/Journal\ Papers/SumResynthesis/data/ordering_scores.csv` |> run

In [ ]:
`git push` |> run

In [ ]:
function jaccard_index(aa::DataStructures.Accumulator,bb::DataStructures.Accumulator)
    sum(intersect(aa,bb))/sum(union(aa,bb))
end

function jaccard_index(aa,bb)
    length(intersect(aa,bb))/length(union(aa,bb))
end

In [ ]:


brown_glove300_raw = load("results/bags/brown_g50D_res.jld", "res")
brown_glove200_raw = load("results/bags/brown_glove200_res.jld", "res")
brown_glove100_raw = load("results/bags/brown_glove100_res.jld", "res")
brown_glove50_raw = load("results/bags/brown_glove50_res.jld", "res")

In [ ]:
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove300_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove200_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove100_raw])

In [ ]:
bag_res = DataFrame()
bag_res[:ground] = [counter(rset[1]) for rset in brown_glove50_raw]
bag_res[:ground_len] = Int[sum(ss) for ss in bag_res[:ground]]
exp_names = ASCIIString[]
for raw_name in ["brown_glove50_raw","brown_glove100_raw", "brown_glove200_raw", "brown_glove300_raw"]
    colname = split(raw_name,"_")[2] 
    push!(exp_names, colname)
    raw = eval(raw_name|>Symbol)
    bag_res[colname*"_actual"|> Symbol] = [counter(rset[2]) for rset in raw]
    #bag_res[colname*"_distance"|> Symbol] = Float64[-1*rset[3] for rset in raw]
end

bag_res

In [ ]:
for exp_name in exp_names
    actual = bag_res[exp_name*"_actual" |> symbol]
    bag_res[exp_name*"_perfect" |> symbol] = actual.==bag_res[:ground]
    bag_res[exp_name*"_jaccard" |> symbol] = map(jaccard_index, actual, bag_res[:ground])
end

In [ ]:

jaccard_cols = Symbol[exp_name*"_jaccard" |> symbol for exp_name in exp_names]
perfect_cols = Symbol[exp_name*"_perfect" |> symbol for exp_name in exp_names]
len_scores = aggregate(bag_res[[:ground_len, jaccard_cols...,perfect_cols...]], :ground_len, mean)
sort!(len_scores)
writetable(OUTPUT_DIR*"selection_len_scores.csv", len_scores)
len_scores

In [ ]:
cum_len_scores = DataFrame()
cum_len_scores[:max_ground_len] = bag_res[:ground_len] |> unique |> sort!
for jaccard_col in jaccard_cols
    cum_len_scores[jaccard_col] = Float64[ begin
            shortish_sents = bag_res[:ground_len] .<= max_len;
            bag_res[shortish_sents, jaccard_col] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end
for perfect_col in perfect_cols
    cum_len_scores[perfect_col] = Float64[ begin
            shortish_sents = bag_res[:ground_len] .<= max_len;
            bag_res[shortish_sents, perfect_col] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end
writetable(OUTPUT_DIR*"selection_cum_len_scores.csv", cum_len_scores)
cum_len_scores 


In [ ]:
function longform(df::DataFrame, index::Symbol, cols::Vector{Symbol}, new_name::Symbol)
    function subdf(col::Symbol)
        df_sub = df[[index, col]]
        rename!(df_sub, col, new_name)
        df_sub[:label] = col
        df_sub
    end
    vcat([subdf(col) for col in cols]...)
end




In [ ]:
using Gadfly

In [ ]:
p=Gadfly.plot(longform(cum_len_scores, :max_ground_len, jaccard_cols, :mean_jaccard_index),
    x="max_ground_len", y="mean_jaccard_index",
    color="label",
    Geom.point
)
#draw(PNG("brown_cumulative_length_score.png", 24cm, 6cm), p)

In [ ]:

p=Gadfly.plot(longform(len_scores, :ground_len, [string(col)*"_mean"|> Symbol for col in jaccard_cols], :mean_jaccard_index),
    x="ground_len", y="mean_jaccard_index",
    color="label",
    Geom.point
)
#draw(PNG("brown_length_score.png", 24cm, 6cm), p)


In [ ]:
bag_res[:ground_len] |> describe 
p=Gadfly.plot(bag_res, x="ground_len",  Geom.histogram)
#draw(PNG("brown_length_dist.png", 24cm, 6cm), p)

In [ ]:
##############################################
# ORDERED
########################################

In [12]:
using PyCall
@pyimport nltk
@pyimport nltk.align.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(candidate,Any[reference], weights)
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


bleu_score (generic function with 1 method)

In [ ]:
bleu_score(["the","comedy","of","manners","of","this","first","a","basis","is","performed","in","1892","."],
           ["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."]
)

In [26]:
#folds50 = load("results/ordered/brown_glove50_ordered_vA1000.jld")
#folds100 = load("results/ordered/brown_glove100_ordered_vA1000.jld")
#folds200 = load("results/ordered/brown_glove200_ordered_vA1000.jld")
folds300_loglikelyhood_A5000_2 = load("results/ordered/brown_glove300_ordered_vA5000_loglikelyhood_2.jld")
folds300_loglikelyhood_A5000 = load("results/ordered/brown_glove300_ordered_vA5000_loglikelyhood.jld")
#folds300_loglikelyhood_A1000 = load("results/ordered/brown_glove300_ordered_vA1000_loglikelyhood.jld")
#folds300 = load("results/ordered/brown_glove300_ordered_vA1000.jld")
#folds_oracle = load("results/ordered/brown_oracle_ordered_vA1000.jld")\

#folds_oracle_loglikelyhood_A5000 = load("results/ordered/brown_oracle_ordered_vA5000_loglikelyhood.jld")
#folds_oracle_loglikelyhood_A5000_2 = load("results/ordered/brown_oracle_ordered_vA5000_loglikelyhood_2.jld")
#folds_oracle_loglikelyhood_A10000_2 = load("results/ordered/brown_oracle_ordered_vA10000_loglikelyhood_2.jld")
#folds_oracle_loglikelyhood_A1000 = load("results/ordered/brown_oracle_ordered_vA1000_Loglikelyhood.jld")
#all_ordered_exp_folds = Any[folds_oracle, folds_oracle_loglikelyhood,folds300,folds300_loglikelyhood]#[folds50,folds100,folds200,folds300,folds_oracle]
all_ordered_exp_folds=Any[folds300_loglikelyhood_A5000,folds300_loglikelyhood_A5000_2]


2-element Array{Any,1}:
 Dict{ByteString,Any}("fold_6"=>Any[(ASCIIString["the","songs","sandburg","sang","often","reminded","listeners","of","songs","of","a","kindred","character","they","knew","entirely","or","in","fragments","."],ASCIIString["songs","of","they","the","songs","a","character","or","sandburg","fragments","entirely","kindred","in",".","reminded","of","often","knew","sang","listeners"],NaN),(ASCIIString["it","will","help","to","determine","the","attitude","of","the","new","members","toward","the","church","."],Any["the","new","members","of","the","attitude","toward","it","will","help","to","determine","the","church","."],2.8747047119667224e-25),(ASCIIString["(","check","on","the","payne","luggage","."],ASCIIString["check","the","(","on",".","luggage","payne"],5.0),(ASCIIString["when","it","was","over",",","eight","of","his","company","were","still","alive","and","all","eight","were","wounded","."],Any["it","when","his","company","were","still","alive",",","were","over","i

In [27]:
function get_bleu_score(folds)
    bleu_scores = map(keys(folds), values(folds)) do fold_name,res
        fold_bleu_scores = Float64[]
        for (reference, candidate, prob) in res
            push!(fold_bleu_scores, bleu_score(candidate, reference))
        end
        fold_bleu_scores
    end
end

get_bleu_score (generic function with 1 method)

In [28]:
attempted_only{T}(fold::Vector{T}) = filter(eap->!isnan(eap[3]), fold)
nongiveup_only{T}(fold::Vector{T}) = filter(eap->0<eap[3]<1, fold)
partial_only{T}(fold::Vector{T}) = filter(eap->eap[3]>1, fold)

attempted_only{T}(folds::Vector{Vector{T}}) = map(attempted_only, folds)
nongiveup_only{T}(folds::Vector{Vector{T}}) = map(nongiveup_only, folds)
partial_only{T}(folds::Vector{Vector{T}}) = map(partial_only, folds)

partial_only (generic function with 2 methods)

In [29]:
bleu_scores = map(all_ordered_exp_folds) do folds
    bleu_scores = get_bleu_score(folds);
    map(mean, bleu_scores) |> mean
end

2-element Array{Any,1}:
 0.104507
 0.100344

In [30]:
bleu_scores_no_giveup = map(all_ordered_exp_folds) do folds
    mean([mean([bleu_score(eap[1], eap[2]) for eap in nongiveup_only(attempted_only(fold))]) for fold in values(folds)])
end

2-element Array{Any,1}:
 0.497722
 0.499132

In [31]:
giveups = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[!(0<eap[3]<1) for eap in attempted_only(fold)]) for fold in values(folds)])
end

2-element Array{Any,1}:
 0.703142
 0.713763

In [32]:
exact_matchs = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[1]==eap[2] for eap in attempted_only(fold)]) for fold in values(folds)])
end

2-element Array{Any,1}:
 0.131246
 0.130618

In [33]:
partials = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[3]>1 for eap in attempted_only(fold)]) for fold in values(folds)])
end

2-element Array{Any,1}:
 0.668093
 0.678843

In [34]:
exact_matchs_no_giveup = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[1]==eap[2] for eap in nongiveup_only(attempted_only(fold))]) for fold in values(folds)])
end

2-element Array{Any,1}:
 0.37672 
 0.388534

In [ ]:
ordering_res = DataFrame(Model=["Oracle", "Oracle_LL", "300d", 300d_LL])#["50d","100d","200d","300d", "Oracle"])
ordering_res[symbol("BLEU Score")]=bleu_scores
ordering_res[symbol("Exact Ordered Matches")]=exact_matchs
ordering_res[symbol("Giveups")]=giveups
ordering_res[symbol("BLEU Score Excluding Giveups")]=bleu_scores_no_giveup
ordering_res[symbol("Exact Ordered Matches Excluding Giveups")]=exact_matchs_no_giveup
#writetable(OUTPUT_DIR*"ordering_scores.csv", ordering_res)
ordering_res

In [ ]:
########

In [ ]:
fold_indexes=load("brown_glove_folds.jld","fold_indexes");

In [ ]:
function fold(data, fold_indexes)
    ret = Dict()
    for (fold_number,fold_iis) in enumerate(fold_indexes)
        ret["fold_$(fold_number)"] = data[fold_iis]
    end
    ret
end

In [ ]:
jaccard_folds_300 = fold(bag_res[:glove300_jaccard], fold_indexes)

In [ ]:
[jaccard_folds_300["fold_6"] bleu_scores300[1]]

In [ ]:
get_bleu_score(folds300,  count_give_ups=true, count_nonattepts=true);